In [15]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd

In [17]:
my_data=[['slashdot','USA','yes',18,'None'],
        ['google','France','yes',23,'Premium'],
        ['digg','USA','yes',24,'Basic'],
        ['kiwitobes','France','yes',23,'Basic'],
        ['google','UK','no',21,'Premium'],
        ['(direct)','New Zealand','no',12,'None'],
        ['(direct)','UK','no',21,'Basic'],
        ['google','USA','no',24,'Premium'],
        ['slashdot','France','yes',19,'None'],
        ['digg','USA','no',18,'None'],
        ['google','UK','no',18,'None'],
        ['kiwitobes','UK','no',19,'None'],
        ['digg','New Zealand','yes',12,'Basic'],
        ['slashdot','UK','no',21,'None'],
        ['google','UK','yes',18,'Basic'],
        ['kiwitobes','France','yes',19,'Basic']]

In [19]:
# Divides a set on a specific column. Can handle numeric or nominal values
def divideset(rows,column,value):
    # Make a function that tells us if a row is in the first group (true) or the second group (false)
    split_function=None
    if isinstance(value,int) or isinstance(value,float):
        split_function=lambda row:row[column]>=value
    else:
        split_function=lambda row:row[column]==value
   
    # Divide the rows into two sets and return them
    set1=[row for row in rows if split_function(row)]
    set2=[row for row in rows if not split_function(row)]
    return (set1,set2)


In [20]:
divideset(my_data,2,'yes')

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['slashdot', 'UK', 'no', 21, 'None']])

In [21]:
def uniquecounts(rows):
    results={}
    for row in rows:
        # The result is the last column
        r=row[len(row)-1]
        if r not in results: results[r]=0
        results[r]+=1
    return results

print(uniquecounts(my_data))

{'None': 7, 'Premium': 3, 'Basic': 6}


In [22]:
def entropy(rows):
    from math import log
    log2=lambda x:log(x)/log(2)  
    results=uniquecounts(rows)
    # Now calculate the entropy
    ent=0.0
    for r in results.keys():
        p=float(results[r])/len(rows)
        ent=ent-p*log2(p)
    return ent


set1,set2=divideset(my_data,3,20)
print entropy(set1), entropy(set2)
print entropy(my_data)

1.44881563573 0.918295834054
1.50524081494


In [23]:
class decisionnode:
    def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
        self.col=col
        self.value=value
        self.results=results
        self.tb=tb # Decision node for true
        self.fb=fb # Decision node for false

In [24]:
def buildtree(rows,scoref=entropy): #rows is the set, either whole dataset or part of it in the recursive call, 
                                    #scoref is the method to measure heterogeneity. By default it's entropy.
    if len(rows)==0: return decisionnode() #len(rows) is the number of units in a set
    current_score=scoref(rows)

    # Set up some variables to track the best criteria
    best_gain=0.0
    best_criteria=None
    best_sets=None
  
    column_count=len(rows[0])-1   #count the # of attributes/columns. 
                                #It's -1 because the last one is the target attribute and it does not count.
    for col in range(0,column_count):
    # Generate the list of all possible different values in the considered column
        global column_values        #Added for debugging
        column_values={}            
        for row in rows:
            column_values[row[col]]=1   
            # Now try dividing the rows up for each value in this column
        for value in column_values.keys(): #the 'values' here are the keys of the dictionnary
            (set1,set2)=divideset(rows,col,value) #define set1 and set2 as the 2 children set of a division
      
            # Information gain
            p=float(len(set1))/len(rows) #p is the size of a child set relative to its parent
            gain=current_score-p*scoref(set1)-(1-p)*scoref(set2) #cf. formula information gain
            if gain>best_gain and len(set1)>0 and len(set2)>0: #set must not be empty
                best_gain=gain
                best_criteria=(col,value)
                best_sets=(set1,set2)
        
    # Create the sub branches   
    if best_gain>0:
        trueBranch=buildtree(best_sets[0])
        falseBranch=buildtree(best_sets[1])
        return decisionnode(col=best_criteria[0],value=best_criteria[1],
                            tb=trueBranch,fb=falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))

In [25]:
tree=buildtree(my_data)

In [28]:
print(tree.col)
print(tree.value)
print(tree.results)
print("----------")
print(tree.tb.col)
print(tree.tb.value)
print(tree.tb.results)
print("----------")
print(tree.tb.tb.col)
print(tree.tb.tb.value)
print(tree.tb.tb.results)
print("----------")
print(tree.tb.fb.col)
print(tree.tb.fb.value)
print(tree.tb.fb.results)
print("----------")
print(tree.tb.fb.tb.col)
print(tree.tb.fb.tb.value)
print(tree.tb.fb.tb.results)
print("----------")
print(tree.tb.fb.fb.col)
print(tree.tb.fb.fb.value)
print(tree.tb.fb.fb.results)

0
google
None
----------
3
21
None
----------
-1
None
{'Premium': 3}
----------
2
yes
None
----------
-1
None
{'Basic': 1}
----------
-1
None
{'None': 1}


In [30]:
print(tree.fb.col)
print(tree.fb.value)
print(tree.fb.results)
print("----------")
print(tree.fb.tb.col)
print(tree.fb.tb.value)
print(tree.fb.tb.results)
print("----------")
print(tree.fb.fb.col)
print(tree.fb.fb.value)
print(tree.fb.fb.results)
print("----------")
print(tree.fb.fb.tb.col)
print(tree.fb.fb.tb.value)
print(tree.fb.fb.tb.results)
print("----------")
print(tree.fb.fb.fb.col)
print(tree.fb.fb.fb.value)
print(tree.fb.fb.fb.results)
print("----------")
print(tree.fb.fb.fb.tb.col)
print(tree.fb.fb.fb.tb.value)
print(tree.fb.fb.fb.tb.results)
print("----------")
print(tree.fb.fb.fb.fb.col)
print(tree.fb.fb.fb.fb.value)
print(tree.fb.fb.fb.fb.results)

0
slashdot
None
----------
-1
None
{'None': 3}
----------
2
yes
None
----------
-1
None
{'Basic': 4}
----------
3
21
None
----------
-1
None
{'Basic': 1}
----------
-1
None
{'None': 3}


In [26]:
def classify(observation,tree):
    if tree.results!=None:
        return tree.results
    else:
        v=observation[tree.col]
        branch=None
        if isinstance(v,int) or isinstance(v,float):
            if v>=tree.value: branch=tree.tb
            else: branch=tree.fb
        else:
            if v==tree.value: branch=tree.tb
            else: branch=tree.fb
        return classify(observation,branch)

In [27]:
classify(['(direct)','USA','yes',5], tree)

{'Basic': 4}

In [31]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas


In [ ]:
dt = DecisionTreeClassifier()

dt.fit()

In [32]:
DecisionTreeClassifier?

In [ ]:
rf = RandomForestClassifier(n_estimators= ) # n_estimators is the no. of decision trees